In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import date, timedelta, datetime
import urllib.request, json 
import os
%matplotlib inline
df_m = pd.read_csv('clean_m_data.csv', index_col = 0, parse_dates = ['date'])
df_m.name = 'Moped Data'
df_c = pd.read_csv('clean_c_data.csv', index_col = 0, parse_dates = ['date'])
df_c.name = 'Car Data'

In [2]:
# https://darksky.net/dev/docs requests I keep my key hidden
# Going back a directory to access darkskyid.txt and save as my_id
os.chdir("..")
my_id = open('darkskyid.txt', 'r').read()
# Going back to original directory
os.chdir("/Users/ethanfuerst/Documents/Coding/mpgdata")

In [3]:
startTime = datetime.now()

# creating a range of dates to get - shoutout date.today()
sdate = date(2019, 1, 1)   # start date - Jan 1 2019
edate = date.today()       # today

delta = edate - sdate       # as timedelta

# creating a list for each day in the range
# using unix time because that's what the darksky api uses
# each date will give me the unix time at 6pm for that day
# the daily_high and daily_low are the parameters that I will use to see how my mpg changes
date_list = []
for i in range(1, delta.days + 1):
    day = sdate + timedelta(days=i)
    date_list.append(int((day - dt.date(1970,1,1)).total_seconds()))

# creating a list for each api call
api_list = []
for i in date_list:
    api_list.append('https://api.darksky.net/forecast/'+my_id+'/30.267153,-97.7430608,'+str(i)+'?exclude=flags,hourly')

# getting the high temp and low temp for each day
date = []
daily_high = []
daily_low = []
for api in api_list:
    with urllib.request.urlopen(api) as url:
        data = json.loads(url.read().decode())
        # keeping day in yyyymmdd format, just like in other dataframes
        date.append(datetime.fromtimestamp(data["currently"]["time"]).strftime("20%y%m%d"))
        daily_high.append(data["daily"]["data"][0]["temperatureHigh"])
        daily_low.append(data["daily"]["data"][0]["temperatureLow"])

# putting all the lists in to a dateframe
df = pd.DataFrame({'date': date, 'daily_high': daily_high, 'daily_low': daily_low})
df['high_mov_avg'] = df['daily_high'].rolling(window=5).mean()
df['low_mov_avg'] = df['daily_low'].rolling(window=5).mean()
print(datetime.now() - startTime)

0:01:17.684914


In [4]:
# date_list 
# df['daily_low'].describe()
df
# change moving average to 3

date  daily_high  daily_low  high_mov_avg  low_mov_avg
0    20190101       44.06      36.17           NaN          NaN
1    20190102       38.24      37.16           NaN          NaN
2    20190103       51.91      32.95           NaN          NaN
3    20190104       66.22      34.10           NaN          NaN
4    20190105       73.18      41.21        54.722       36.318
..        ...         ...        ...           ...          ...
249  20190907       98.60      75.53        98.522       74.448
250  20190908       97.95      76.72        98.606       75.004
251  20190909       96.45      75.52        98.364       75.860
252  20190910       95.90      71.61        97.636       75.188
253  20190911       94.56      72.31        96.692       74.338

[254 rows x 5 columns]